
# ChatGPT API Natural Language Magic: The Gathering Card Generator

by Max Woolf ([@minimaxir](https://twitter.com/minimaxir))

This Colab Notebook demonstrates how ChatGPT can be used to output structured JSON relatively consistently, which can be used for downstream applications such as using the data for template formatting.

This ChatGPT API sentiment analyzer requires an OpenAI account with a payment method attached to it/a free trial, and an [OpenAI API Key](https://platform.openai.com/account/api-keys). Running the setup cells by **mousing over the cells and pressing the Play button** will prompt you to input a key from that link and press Enter; it will not be saved to the Notebook.

If generated one at a time (e.g. `Create a Magic card`), each card takes ~280 ChatGPT tokens to generate, or ~$0.55 per 1,000 cards generated. If 5 cards are generated at a time (e.g. `Create five variations of Magic cards`), it takes ~123 tokens to generate a card on average, about half the cost, and therefore this Notebook will use that default.

## Setup

In [1]:
!pip install -q openai rich ujson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [2]:
import openai
import os
import re
from rich.console import Console
import re
import getpass
import json
from jinja2 import Template
import sys
from tqdm.auto import tqdm
from google.colab import files
import ujson

api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


In [3]:
# card_extract_pattern = r'(\{"name":.*rarity.*\})'
card_extract_pattern = r'(\{.*\})'
def color_rarity(value, to_notebook=True):
    value = value.lower()
    # Can only display colors if printing to the notebook
    if to_notebook:
        if "mythic" in value:
            value = f"[bright_magenta]{value}[/bright_magenta]"
        elif "rare" in value:
            value = f"[bright_blue]{value}[/bright_blue]"
        elif "uncommon" in value:
            value = f"[bright_green]{value}[/bright_green]"
    return value

TEMPLATE = Template(
    """{{ c.name }}{% if c.manaCost %}  {{ c.manaCost }}{% endif %}
{{ c.type }}{% if c.text %}
{{ c.text }}{% endif %}{% if c.flavorText %}
{{ c.flavorText }}{% endif %}{% if c.pt %}
{{ c.pt }}{% elif c.loyalty %}
Loyalty: {{ c.loyalty }}{% endif %}
{{ c.rarity }}"""
)


def render_card(card_dict):
    return TEMPLATE.render(c=card_dict)

In [4]:
system = """You are an assistant who works as a Magic: The Gathering card designer. Create cards that are in the following card schema and JSON format. OUTPUT MUST FOLLOW THIS CARD SCHEMA AND JSON FORMAT. DO NOT EXPLAIN THE CARD. The output must also follow the Magic "color pie".

{"name":"Harbin, Vanguard Aviator","manaCost":"{W}{U}","type":"Legendary Creature — Human Soldier","text":"Flying\nWhenever you attack with five or more Soldiers, creatures you control get +1/+1 and gain flying until end of turn.","flavorText":"\"Yotia is my birthright, father. Let me fight for it.\"","pt":"3/2","rarity":"rare"}"""

def generate_magic_cards(prompt):
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        # stop="<|DONE|>",
        max_tokens=1500,  # sanity limit
        temperature=0.8,  # increasing temp higher than 0.8 may cause non-JSON output
    )
    # print(r["usage"])
    return r["choices"][0]["message"]["content"]


## Generate the Card!

Some example inputs:

- Create five variations of five-color Mythic Rare Artifact Magic cards.
- Create five variations of Magic cards with the text "You win the match"
- Create five variations of Planeswalker Magic cards with the subtype "Bob".
- Create five variations of Magic cards with "What's updog?" as the card flavor text.
- Create five variations of Magic cards named "Darth Vader" with a mana cost of atleast ten.
- Create five variations of multicolor Magic cards based on the War of 1812.
- Create five variations of Sorcery Magic cards based on Final Fantasy VII.
- Create ten variations of Magic cards based on a crossover between the War of 1812 and Final Fantasy VII.
- Create ten variations of Magic cards based on a crossover between Twitter and TikTok.

In addition to cost savings, the "five variations" framing ensures all generations are distinct.

All generations are saved to a text file in the Notebook sidebar, which can be downloaded locally.

`n_iterations` will rerun the same prompt _n_ amount of times: good if you want to generate a large batch of cards to a file.

In [8]:
from ujson import JSONDecodeError
from datetime import datetime
num_repeat_to_file = 10
output_width = 60


prompt = "Create ten variations of Magic cards based on Spongebob Squarepants and ancient Roman history." #@param {type:"string"}
to_file = False  # not quite working yet
n_iterations = 1 #@param {type:"slider", min:1, max:10, step:1}

console = Console(width=60, record=True)

for _ in tqdm(range(n_iterations)):
    success = True
    outputs = generate_magic_cards(prompt)

    cards = re.split(r"\n{2,}", outputs)
    processed_cards = []

    for card in cards:
        try:
            card = re.search(card_extract_pattern, card, re.S)
            if card is not None:
                card = str(card.group(0))
                card = card.replace('"",', '",')  # handle double quotes just in case
                card = ujson.loads(card)
                card["name"] = f"[bold]{card['name']}[/bold]"
                if "flavorText" in card:
                    card["flavorText"] = f"[italic]{card['flavorText']}[/italic]"
                card["rarity"] = color_rarity(card["rarity"])
                if "power" in card and "toughness" in card:
                    card["pt"] = f'{card["power"]}/{card["toughness"]}'
                processed_cards.append(render_card(card))
        except JSONDecodeError as e:
            console.print(f"[bright_red]<JSON Parsing Failed! 😭>\nCard output: {card}[/bright_red]", highlight=False)
            success = False
            continue
        except KeyError as e:
            console.print(f"[bright_red]<Missing a required card attribute! 😭>\nCard output: {card}[/bright_red]", highlight=False)
            success = False
            continue

    if len(processed_cards) == 0:
        success = False
        console.print(f"[bright_red]<No cards generated! 😭>\nChatGPT output: {outputs}[/bright_red]", highlight=False)

    if success:
        console.print(("\n" + "─" * output_width + "\n").join(processed_cards), highlight=False)
        console.print("─" * output_width, highlight=False)
        # console.print(f"ChatGPT Output:\n{outputs}", highlight=False)

console.save_text(f"cards_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

  0%|          | 0/1 [00:00<?, ?it/s]

Spongebob, Gladiator of Bikini Bottom  {2}{R}
Creature — Sponge Warrior
First strike, protection from blue
Whenever Spongebob, Gladiator of Bikini Bottom deals combat 
damage to a player, put a +1/+1 counter on it.
{R}: Spongebob, Gladiator of Bikini Bottom gains haste until
end of turn.
I'm ready!
2/2
rare
────────────────────────────────────────────────────────────
Patrick, Emperor of the IJKL Empire  {3}{W}{U}
Legendary Creature — Starfish Emperor
Vigilance, protection from black
Whenever an opponent casts a spell, put a +1/+1 counter on 
Patrick, Emperor of the IJKL Empire.
{4}{W}{U}: Exile target creature an opponent controls until 
Patrick, Emperor of the IJKL Empire leaves the battlefield.
I am the ruler of the seven seas!
3/4
mythic
────────────────────────────────────────────────────────────
Squidward, Tribune of the Krusty Krab  {1}{W}{B}
Legendary Creature — Squid Advisor
Deathtouch
Whenever a creature an opponent controls dies, put a +1/+1 
counter on Squidward, Tribune of the Krusty Krab.
{4}{B}: Target creature gets -2/-2 until end of turn.
I am not a Krusty Krab.
2/4
rare
────────────────────────────────────────────────────────────
Sandy, Praetor of Texas  {2}{G}{W}
Legendary Creature — Squirrel Soldier
Vigilance, reach
Whenever Sandy, Praetor of Texas blocks or becomes blocked 
by a creature, that creature doesn’t untap during its 
controller's next untap step.
{3}{G}{W}: Sandy, Praetor of Texas gets +2/+2 until end of 
turn.
Y'all best not mess with Texas!
2/3
rare
────────────────────────────────────────────────────────────
Mr. Krabs, Merchant of the Amphoras  {3}{W}{B}
Legendary Creature — Crab Merchant
Lifelink, protection from red
Whenever a creature an opponent controls dies, you gain 1 
life and draw a card.
{4}{W}{B}: Sacrifice a permanent, then gain life equal to 
its converted mana cost.
Money, money, money!
3/3
mythic
────────────────────────────────────────────────────────────
Plankton, Conspirator of the Chum Bucket  {1}{U}{B}
Legendary Creature — Plankton Advisor
Menace
Whenever a player draws a card, each opponent loses 1 life 
and you gain 1 life.
{2}{U}{B}: Target player discards a card.
I went to college!
1/3
rare
────────────────────────────────────────────────────────────
Gary, Commander of the Snail Legion  {3}{W}{U}
Legendary Creature — Snail Soldier
Hexproof, first strike
Whenever a creature an opponent controls attacks, put a 
+1/+1 counter on Gary, Commander of the Snail Legion.
{3}{W}{U}: Tap target creature an opponent controls.
Meow!
2/4
rare
────────────────────────────────────────────────────────────
Pearl, Queen of the Tritons  {4}{U}{U}
Legendary Creature — Whale Noble
Islandwalk, trample
Whenever Pearl, Queen of the Tritons deals combat damage to 
a player, draw a card.
{5}{U}{U}: Until end of turn, target creature gets +3/+3 and
islandwalk.
I'm not just a kid!
4/4
mythic
────────────────────────────────────────────────────────────
SpongeGar, Scourge of the Prehistoric Age  {2}{B}{G}
Legendary Creature — Sponge Caveman
Trample
Whenever SpongeGar, Scourge of the Prehistoric Age deals 
combat damage to a player, put a +1/+1 counter on it.
{3}{B}{G}: SpongeGar, Scourge of the Prehistoric Age gains 
deathtouch until end of turn.
Meep!
2/2
rare
────────────────────────────────────────────────────────────
Karen, Oracle of the Digital Realm  {2}{U}{B}
Legendary Creature — Computer Advisor
Flying
Whenever you cast an instant or sorcery spell, scry 1 and 
target creature gets -1/-1 until end of turn.
{3}{U}{B}: Draw a card, then discard a card.
Welcome to the Chum Bucket, where we serve Krabby Patties!
2/3
rare

────────────────────────────────────────────────────────────

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
